### 你想构造一个可接受任意数量参数的函数

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

# Sample use
print(avg(1, 2))
print(avg(1, 2, 3, 4))

1.5
2.5


In [2]:
def anyargs(*args, **kwargs):
    print(args) # A tuple
    print(kwargs) # A dict

In [4]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                name=name,
                attrs=attr_str,
                value=html.escape(value))
    return element

# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)

# Creates '<p>&lt;spam&gt;</p>'
#make_element('p', '<spam>')

'<item size="large" quantity="6">Albatross</item>'

### 你希望函数的某些参数强制使用关键字参数传递
将强制关键字参数放到某个*参数或者单个*后面就能达到这种效果
def recv(maxsize, *, block):
    'Receives a message'
    pass

recv(1024, True) # TypeError
recv(1024, block=True) # Ok

In [9]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

#minimum(1, 5, 2, -5, 10) # Returns -5
minimum(1, 5, 2, -5, 10, clip=0) # Returns 0

0

### 你写好了一个函数，然后想为这个函数的参数增加一些额外的信息，这样的话其他使用者就能清楚的知道这个函数应该怎么使用。

In [10]:
def add(x:int, y:int) -> int:
    return x + y

print(add(3,4))

7


In [11]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



In [12]:
add.__annotations__

{'return': int, 'x': int, 'y': int}

### 你希望构造一个可以返回多个值的函数

In [13]:
def myfun():
     return 1, 2, 3

a,b,c = myfun()
print(a)
print(b)

1
2


### 你想定义一个函数或者方法，它的一个或多个参数是可选的并且有一个默认值

In [14]:
def spam(a, b=42):
    print(a, b)

spam(1) # Ok. a=1, b=42
spam(1, 2) # Ok. a=1, b=2

1 42
1 2


In [15]:
names = ['David Beazley', 'Brian Jones', 'Raymond Hettinger', 'Ned Batchelder']
ll = sorted(names, key=lambda name: name.split()[-1].lower())
print(ll)

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']


### 你有一个除 __init__() 方法外只定义了一个方法的类。为了简化代码，你想将它转换成一个函数

from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

# Example use. Download stock data from yahoo
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Example use
yahoo = urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

### 你想要扩展函数中的某个闭包，允许它能访问和修改函数的内部变量。

In [18]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)

    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()
f()
f.set_n(10)
f.get_n()

n= 0


10

### 当你编写使用回调函数的代码的时候，担心很多小函数的扩张可能会弄乱程序控制流。 你希望找到某个方法来让代码看上去更像是一个普通的执行序列。

In [23]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)

    # Invoke the callback with the result
    callback(result)

from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

In [24]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [25]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye
